In [7]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:default:

In [8]:
%%sql
select 'hola mundo' 

,'hola mundo'
0,hola mundo


In [9]:
%%sql
create or replace table competencia_02 as 
select 
    *
from read_csv_auto('../datasets/competencia_02_crudo.csv')

,Success


In [10]:
%%sql 
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = 'competencia_02';


,column_name,data_type
0,numero_de_cliente,BIGINT
1,foto_mes,BIGINT
2,active_quarter,BIGINT
3,cliente_vip,BIGINT
4,internet,BIGINT
...,...,...
149,Visa_fechaalta,BIGINT
150,Visa_mconsumototal,DOUBLE
151,Visa_cconsumos,BIGINT
152,Visa_cadelantosefectivo,BIGINT


In [11]:
%%sql
create or replace table competencia_02 as 

with max_db as 
    (
        select max(foto_mes) as max_foto_mes_db
        from competencia_02
    )
select c.*, max_db.max_foto_mes_db, max_numero_de_cliente.max_foto_mes, c.foto_mes,
    case
        when max_db.max_foto_mes_db = max_numero_de_cliente.max_foto_mes then 'continua'
        when CAST(SUBSTRING(max_numero_de_cliente.max_foto_mes, 1, 4) AS INTEGER) * 12 + CAST(SUBSTRING(max_numero_de_cliente.max_foto_mes, 5, 2) AS INTEGER) -
           (CAST(SUBSTRING(c.foto_mes, 1, 4) AS INTEGER) * 12 + CAST(SUBSTRING(c.foto_mes, 5, 2) AS INTEGER)) =  0 then 'baja+1'
        when CAST(SUBSTRING(max_numero_de_cliente.max_foto_mes, 1, 4) AS INTEGER) * 12 + CAST(SUBSTRING(max_numero_de_cliente.max_foto_mes, 5, 2) AS INTEGER) -
           (CAST(SUBSTRING(c.foto_mes, 1, 4) AS INTEGER) * 12 + CAST(SUBSTRING(c.foto_mes, 5, 2) AS INTEGER)) == 1 then 'baja+2'
         when CAST(SUBSTRING(max_numero_de_cliente.max_foto_mes, 1, 4) AS INTEGER) * 12 + CAST(SUBSTRING(max_numero_de_cliente.max_foto_mes, 5, 2) AS INTEGER) -
           (CAST(SUBSTRING(c.foto_mes, 1, 4) AS INTEGER) * 12 + CAST(SUBSTRING(c.foto_mes, 5, 2) AS INTEGER)) > 1 then 
            concat('baja+', CAST(SUBSTRING(max_numero_de_cliente.max_foto_mes, 1, 4) AS INTEGER) * 12 + CAST(SUBSTRING(max_numero_de_cliente.max_foto_mes, 5, 2) AS INTEGER) -
                            (CAST(SUBSTRING(c.foto_mes, 1, 4) AS INTEGER) * 12 + CAST(SUBSTRING(c.foto_mes, 5, 2) AS INTEGER)))
    end as clase_ternaria, 
    CAST(SUBSTRING(max_numero_de_cliente.max_foto_mes, 1, 4) AS INTEGER) * 12 + CAST(SUBSTRING(max_numero_de_cliente.max_foto_mes, 5, 2) AS INTEGER) -
        (CAST(SUBSTRING(c.foto_mes, 1, 4) AS INTEGER) * 12 + CAST(SUBSTRING(c.foto_mes, 5, 2) AS INTEGER)) as valor
from competencia_02 c
    cross join max_db
    left join 
        (
            select numero_de_cliente, max(foto_mes) as max_foto_mes
            from competencia_02
            group by numero_de_cliente
        ) max_numero_de_cliente on max_numero_de_cliente.numero_de_cliente = c.numero_de_cliente 

,Success


In [50]:
%%sql

WITH MesesPossiveis AS (
    SELECT DISTINCT numero_de_cliente, foto_mes
    FROM competencia_02
)

SELECT DISTINCT numero_de_cliente
FROM (
    SELECT
        numero_de_cliente,
        foto_mes,
        LAG(foto_mes, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS mes_anterior
    FROM MesesPossiveis
) subquery
WHERE (
    (CAST(foto_mes AS INT) - CAST(mes_anterior AS INT)) != 1
    AND (
        (CAST(foto_mes AS INT) = 1 AND CAST(mes_anterior AS INT) != 12)
        OR (CAST(foto_mes AS INT) = 12 AND CAST(mes_anterior AS INT) != 11)
    )
)


,numero_de_cliente


In [48]:
%%sql
select * 
from competencia_02
where numero_de_cliente = 29425049
order by numero_de_cliente, foto_mes


,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,max_foto_mes_db,max_foto_mes,foto_mes:1,clase_ternaria,valor
0,29425049,201901,1,0,1,48,263,1058.40,12310.28,181.95,...,5215,216.03,3,0,11.73,202107,202107,201901,continua,30
1,29425049,201902,1,0,1,48,264,752.10,12071.65,-30.05,...,5243,217.00,3,0,11.73,202107,202107,201902,continua,29
2,29425049,201903,1,0,1,48,265,1025.00,11710.99,-36.03,...,5274,217.09,3,0,11.73,202107,202107,201903,continua,28
3,29425049,201904,1,0,1,48,266,735.28,11278.01,10.52,...,5304,217.01,3,0,11.73,202107,202107,201904,continua,27
4,29425049,201905,1,0,1,48,267,0.00,0.00,0.00,...,5335,251.37,3,0,11.73,202107,202107,201905,continua,26
5,29425049,201906,1,0,1,48,268,774.34,10971.26,-60.52,...,5365,251.37,3,0,11.73,202107,202107,201906,continua,25
6,29425049,201907,1,0,1,48,269,1165.90,11038.55,112.44,...,5396,281.84,3,0,11.73,202107,202107,201907,continua,24
7,29425049,201908,1,0,1,48,270,789.32,10898.76,-14.20,...,5427,282.86,3,0,11.73,202107,202107,201908,continua,23
8,29425049,201909,1,0,1,48,271,385.45,10484.83,-310.29,...,5457,282.86,3,0,11.73,202107,202107,201909,continua,22
9,29425049,201910,1,0,1,48,272,0.00,0.00,0.00,...,5488,282.86,3,0,11.73,202107,202107,201910,continua,21


In [52]:
%%sql
COPY competencia_02 TO '../datasets/competencia_02.csv' (FORMAT CSV, HEADER)

RuntimeError: Query interrupted